In [21]:
!pip install shap

     |████████████████████████████████| 358kB 4.9MB/s 
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491604 sha256=0b11178bcf4c61c87a776df07ca4530bedc1dde9b060d0fb1806856b2e2e752f
  Stored in directory: /root/.cache/pip/wheels/15/27/f5/a8ab9da52fd159aae6477b5ede6eaaec69fd130fa0fa59f283
Successfully built shap


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from google.colab import files
import pandas as pd
import json
import re
import ast
import shap

In [2]:
def process_spectra(file_obj, file_name):
    file_string = str(file_obj[file_name])

    return file_string.split("\\n")

In [3]:
def process_matrix(file_obj, file_name):
    file_string = str(file_obj[file_name])

    result = file_string.split("\\n")
    result[0] = result[0].replace("b'", "")

    for i in range(len(result)):
        result[i] = result[i].split(" ")
        result[i] = [int(x) if x.isdigit() else x for x in result[i]]
        lastel = result[i][-1]
        if lastel == "+":
            result[i][-1] = "PASS"
        elif lastel == "-":
            result[i][-1] = "FAIL"
    
    return result

In [4]:
def make_line_in_txt_shap(shap_value_exp_obj_instance, instance_name):
    feature_indexes = []
    new_list = []
    for i in range(len(df.iloc[instance_name])):
        if df.iloc[instance_name][i] == 1:
            feature_indexes.append(i)
    for i in feature_indexes:
        new_list.append(";".join([list(x.columns)[i], str(shap_value_exp_obj_instance[i].values)]))
    return new_list    

In [28]:
def write_output_file(filename, obj, is_reverse=False):
    with open(filename, 'w') as file:
        sorted_dict = sorted(obj.items(), key=lambda x: x[1], reverse=is_reverse)
        for i in sorted_dict:
            file.write(";".join([i[0], str(i[1])]) + "\n")

In [8]:
spectra_file = files.upload()
matrix_file = files.upload()

Saving spectra to spectra (1)


Saving matrix to matrix


In [9]:
spectra_list = process_spectra(spectra_file, list(spectra_file.keys())[0])
spectra_list[-1] = "Pass/Fail"
spectra_list[0] = spectra_list[0].replace("b'", "")

In [10]:
matrix_list = process_matrix(matrix_file, list(matrix_file.keys())[0])

In [11]:
matrix_list.pop()

["'"]

In [12]:
df = pd.DataFrame(data=matrix_list, columns=spectra_list)

In [13]:
df

,org.apache.commons.math3.util.FastMath$ExpFracTable#3744,org.apache.commons.math3.util.FastMath$ExpFracTable#3770,org.apache.commons.math3.util.FastMath$ExpFracTable#3771,org.apache.commons.math3.util.FastMath$ExpFracTable#3773,org.apache.commons.math3.util.FastMath$lnMant#3777,org.apache.commons.math3.util.FastMath$lnMant#3791,org.apache.commons.math3.util.FastMath$lnMant#3793,org.apache.commons.math3.util.FastMath#353,org.apache.commons.math3.util.FastMath#97,org.apache.commons.math3.util.FastMath#115,org.apache.commons.math3.util.FastMath#128,org.apache.commons.math3.util.FastMath#141,org.apache.commons.math3.util.FastMath#160,org.apache.commons.math3.util.FastMath#179,org.apache.commons.math3.util.FastMath#198,org.apache.commons.math3.util.FastMath#218,org.apache.commons.math3.util.FastMath#237,org.apache.commons.math3.util.FastMath#256,org.apache.commons.math3.util.FastMath#277,org.apache.commons.math3.util.FastMath#285,org.apache.commons.math3.util.FastMath#288,org.apache.commons.math3.util.FastMath#365,org.apache.commons.math3.util.FastMath#366,org.apache.commons.math3.util.FastMath#368,org.apache.commons.math3.util.FastMath#369,org.apache.commons.math3.util.FastMath#370,org.apache.commons.math3.util.FastMath#379,org.apache.commons.math3.util.FastMath#387,org.apache.commons.math3.util.FastMath#388,org.apache.commons.math3.util.FastMath#396,org.apache.commons.math3.util.FastMath#397,org.apache.commons.math3.util.FastMath#399,org.apache.commons.math3.util.FastMath#400,org.apache.commons.math3.util.FastMath#402,org.apache.commons.math3.util.FastMath#404,org.apache.commons.math3.util.FastMath#405,org.apache.commons.math3.util.FastMath#407,org.apache.commons.math3.util.FastMath#408,org.apache.commons.math3.util.FastMath#410,org.apache.commons.math3.util.FastMath#414,...,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#164,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#166,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#218,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#260,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#261,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#263,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#264,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#266,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#268,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#269,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#272,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#275,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#276,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#277,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#279,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#280,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#281,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#284,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#285,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#291,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#302,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#321,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#322,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#324,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#325,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#326,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#340,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#353,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#354,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#357,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#372,org.apache.commons.math3.analysis.solvers.UnivariateSolverUtils#373,org.apache.commons.m

In [14]:
x = df[spectra_list[:len(spectra_list)-1]]
y = df["Pass/Fail"]

In [15]:
fails = []
passes = []
i = 0
for result in y:
    if result == "FAIL":
        fails.append(i)
    else:
        passes.append(i)
    i += 1 

In [16]:
if len(fails) == 1:
    df = df.append(df.iloc[fails[0]])
    x = df[spectra_list[:len(spectra_list)-1]]
    y = df["Pass/Fail"]

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=1)

In [18]:
model = RandomForestClassifier(n_estimators=100, bootstrap=True, max_features='sqrt', random_state=1)

In [19]:
model.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [23]:
shap_to_txt = {}

shap_values = shap.TreeExplainer(model).shap_values(x)

for i in fails:
    shap_value_exp_obj = shap.Explanation(shap_values[0][i], feature_names=list(x.columns))
    shap_to_txt[x.iloc[i].name] = []
    shap_to_txt[x.iloc[i].name] += make_line_in_txt_shap(list(shap_value_exp_obj), i)

In [24]:
shap_lines_output_val = {}

for i in shap_to_txt:
    for j in shap_to_txt[i]:
        feature_and_value = j.split(";")
        feature = feature_and_value[0]
        value =  float(feature_and_value[1])
        if feature not in shap_lines_output_val:
            shap_lines_output_val[feature] = []
        shap_lines_output_val[feature].append(value)

In [25]:
shap_means = {}
shap_max = {}
shap_min = {}

for i in shap_lines_output_val:
    if i not in shap_means:
        shap_means[i] = []
    if i not in shap_min:
        shap_min[i] = shap_lines_output_val[i][0]
    if i not in shap_max:
        shap_max[i] = shap_lines_output_val[i][0]
    for j in shap_lines_output_val[i]:
        shap_means[i].append(j)
        if abs(shap_max[i]) < abs(j):
            shap_max[i] = j
        if abs(shap_min[i]) > abs(j):
            shap_min[i] = j

for i in shap_means:
    shap_means[i] = sum(shap_means[i]) / len(shap_means[i])

In [26]:
with open("Math-7_shap_results.txt", "w") as file:
    for i in shap_to_txt:
        file.write(str(i) + "\n")
        for j in range(len(shap_to_txt[i])):
            file.write(shap_to_txt[i][j] + "\n")

In [29]:
write_output_file('Math-7_shap_mean.txt', shap_means, is_reverse=True)
write_output_file('Math-7_shap_min.txt', shap_min, is_reverse=True)
write_output_file('Math-7_shap_max.txt', shap_max, is_reverse=True)